In [ ]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage


d:\02_ Kaustubh\LangGraph\venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [3]:
load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

In [4]:
# generator_llm , evaluat, optmise 
generate_tweet_llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)
evaluate_tweet_llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)
optimize_tweet_llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

In [5]:
class TweetEvalutaion(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="The sentiment of the tweet")
    feedback: str = Field(description="A brief explanation of the sentiment")

In [6]:
tweet_evaluation = evaluate_tweet_llm.with_structured_output(TweetEvalutaion)


In [7]:
class TweetGeneration(TypedDict):
    topic: str 
    tweet: str
    evaluation: Literal["approved", "rejected"]
    feedback:str
    iterations:int
    max_iterations:int
    tweet_generation_history: list[str]
    feedback_history: list[str]

    

    

In [ ]:
def generate_tweet(state: TweetGeneration) -> TweetGeneration:
    message = [
        SystemMessage(content="You are a creative social media manager."),
        HumanMessage(content=f"""
            Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

            Rules:
            - Do NOT use question-answer format.
            - Max 280 characters.
            - Use observational humor, irony, sarcasm, or cultural references.
            - Think in meme logic, punchlines, or relatable takes.
            - Use simple, day to day english
            """)
    ]
    tweet = generate_tweet_llm(message).content
    return tweet